In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Input, Dense, Activation, Dropout, LeakyReLU, Softmax, ReLU
from tensorflow.keras import Model


In [0]:
import os,sys,inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
import process_data

from google.colab import files

In [0]:
def convert_coo_to_sparse_tensor(L):
    indices = np.column_stack((L.row, L.col))
    L = tf.SparseTensor(indices, L.data.astype('float32'), L.shape)
    L = tf.sparse.reorder(L)
    return L

In [0]:
# Load dataset from local

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
 

In [65]:
# Dataset loading

A, X, Y, train_idx, val_idx, test_idx = process_data.load_data("cora")
X = process_data.preprocess_features(X)
# A = convert_coo_to_sparse_tensor(A.tocoo())
A = A + np.eye(A.shape[0])  # Add self-loops

x_train, y_train = X[train_idx], Y[train_idx]
x_val, y_val = X[val_idx], Y[val_idx]
x_test, y_test = X[test_idx], Y[test_idx]

idx = np.zeros(shape=(X.shape[0],))
idx[train_idx] = 1
train_idx = idx
idx = np.zeros(shape=(X.shape[0],))
idx[val_idx] = 1
val_idx = idx
idx = np.zeros(shape=(X.shape[0],))
idx[test_idx] = 1
test_idx = idx

print("x_train\t", x_train.shape)
print("y_train\t", y_train.shape)
print("x_val\t", x_val.shape)
print("y_val\t", y_val.shape)
print("x_test\t", x_test.shape)
print("y_test\t", y_test.shape)

# mask = tf.equal(A.indices[:, 0], 0)
# print(mask)
# tf.boolean_mask(A.indices, mask)

# A_train = tf.sparse.to_dense(tf.sparse.slice(A,
#                                              start=[0,0],
#                                              size=[len(train_idx), len(train_idx)]))
# A_val = tf.sparse.to_dense(tf.sparse.slice(A,
#                                            start=[len(train_idx), len(train_idx)],
#                                            size=[len(val_idx), len(val_idx)]))
# A_test = tf.sparse.to_dense(tf.sparse.slice(A,
#                                             start=[len(train_idx)+len(val_idx), len(train_idx)+len(val_idx)],
#                                             size=[len(test_idx), len(test_idx)]))

# A_train = A[0:len(train_idx),
#             0:len(train_idx)]
# A_val = A[len(train_idx):len(train_idx)+len(val_idx),
#                           len(train_idx):len(train_idx)+len(val_idx)]
# A_test = A[len(train_idx)+len(val_idx):len(train_idx)+len(val_idx)+len(test_idx),
#            len(train_idx)+len(val_idx):len(train_idx)+len(val_idx)+len(test_idx)]

(2708, 2708)
(2708, 1433)
x_train	 (140, 1433)
y_train	 (140, 7)
x_val	 (500, 1433)
y_val	 (500, 7)
x_test	 (1000, 1433)
y_test	 (1000, 7)


In [0]:
# x = np.random.rand(10,1)
# y = np.random.rand(10,1)

# # x = np.reshape(x, (1, 1, 2))
# # y = np.reshape(y, (1, 1, 2))

# # x = np.transpose(x)
# y = np.transpose(y)

# print(x)
# print(y)

# z = x + y
# z

In [0]:
### ONE INPUT VERSION ###


# Create attention

class Attention(Layer):
  
  def __init__(self, F_1, A, train_idx, **kwargs):
    self.F_1 = F_1
    self.train_idx = len(train_idx)
    self.A = tf.sparse.slice(A, start=[0,0], size=[self.train_idx, self.train_idx])
    self.A = tf.sparse.to_dense(self.A)
    super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
    F = input_shape[-1]
    self.W = self.add_weight(name='W', 
                             shape=(self.F_1, F),
                             initializer='uniform',
                             trainable=True)

    self.a = self.add_weight(name='a', 
                             shape=(2, self.F_1, 1),
                             initializer='uniform',
                             trainable=True)

    super(Attention, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs

    x_features = X @ tf.transpose(self.W)
#     x_features = (N, F_1)
    att_self = x_features @ self.a[0]
    att_neighbours = x_features @ self.a[1]
#     both att = (N, 1)
    att = att_self + tf.transpose(att_neighbours)
#     att = (N, N)
    att = LeakyReLU(alpha=0.2)(att)
  
    mask = -10e9 * (1.0 - self.A)
    att_masked = att + mask
#     att_masked = att * A
    dense = Softmax(axis=0)(att_masked)
#     dense = (N, N)
    dense = dense @ x_features
#     dense = (N, F_1)

    return dense

  def compute_output_shape(self, input_shape):
    return (input_shape, self.output_dim)
    


# Create Gaussian (kernel) layer

class GAT(Layer):
  
  def __init__(self, F_1, K, A, train_idx, last=False, **kwargs):
    self.F_1 = F_1
    self.K = K
    self.train_idx = train_idx
    self.A = A
    self.last = last
    self.attentions = []
    for k in range(self.K):
      attention = Attention(self.F_1, A, train_idx)
      self.attentions.append(attention)
    super(GAT, self).__init__(**kwargs)
  
  def build(self, input_shape):

    super(GAT, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs

    attentions = []
    for k in range(self.K):
      attention = self.attentions[k](X)
      attentions.append(attention)

    if self.last:
#       h = (N, F_1)
      h = tf.reduce_mean(tf.stack(attentions), axis=0)
      h = Softmax()(h)
    else:
#       h = (N, F_1*k)
      h = tf.concat(attentions, axis=1)
      h = ReLU()(h)
    
    return h

  def compute_output_shape(self, input_shape):
    return (input_shape, self.output_dim)

In [0]:
print(x_train.shape)
h = GAT(8, 8, A=A, train_idx=train_idx, last=False, input_shape=(X.shape[-1],))(x_train)
h = GAT(7, 2, A=A, train_idx=train_idx, last=True, input_shape=(h.shape[-1],))(h)


In [0]:
### ONE INPUT VERSION ###

epochs=1000
batch_size = x_train.shape[0]
n_classes = 7
F_1 = 8
K = 8

model = Sequential()
model.add(GAT(F_1=F_1, K=K, A=A, train_idx=train_idx, last=False, input_shape=(x_train.shape[-1],)))
model.add(GAT(F_1=n_classes, K=1, A=A, train_idx=train_idx, last=True))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print(model.summary())
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False)

In [0]:
### TWO INPUT VERSION ###


# Create attention

class Attention(Layer):
  
  def __init__(self, F_1, **kwargs):
    self.F_1 = F_1
    super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
    F = input_shape[0][-1]
    self.W = self.add_weight(name='W', 
                             shape=(self.F_1, F),
                             initializer='uniform',
                             trainable=True)

    self.a = self.add_weight(name='a', 
                             shape=(2, self.F_1, 1),
                             initializer='uniform',
                             trainable=True)

    super(Attention, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]
    dropout = 0.3

    x_features = X @ tf.transpose(self.W)
#     x_features = (N, F_1)
    att_self = x_features @ self.a[0]
    att_neighbours = x_features @ self.a[1]
#     both att = (N, 1)
    att = att_self + tf.transpose(att_neighbours)
#     att = (N, N)
    att = LeakyReLU(alpha=0.2)(att)

    mask = -10e9 * (1.0 - A)
    att_masked = att + mask
#     att_masked = att * A
    dense = Softmax(axis=0)(att_masked)
  
#     dense = Dropout(dropout)(dense)
#     x_features = Dropout(dropout)(x_features)
    
#     dense = (N, N)
    dense = dense @ x_features
#     dense = (N, F_1)

    return dense

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.output_dim)
    


# Create Gaussian (kernel) layer

class GAT(Layer):
  
  def __init__(self, F_1, K, last=False, **kwargs):
    self.F_1 = F_1
    self.K = K
    self.last = last
    self.attentions = []
    for k in range(self.K):
      attention = Attention(self.F_1)
      self.attentions.append(attention)
    super(GAT, self).__init__(**kwargs)
  
  def build(self, input_shape):

    super(GAT, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]

    attentions = []
    for k in range(self.K):
      attention = self.attentions[k]([X, A])
      attentions.append(attention)

    if self.last:
#       h = (N, F_1)
      h = tf.reduce_mean(tf.stack(attentions), axis=0)
      h = Softmax()(h)
    else:
#       h = (N, F_1*k)
      h = tf.concat(attentions, axis=1)
      h = ReLU()(h)
    return h

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.output_dim)

In [67]:
### TWO INPUT VERSION ###

epochs=300
batch_size = X.shape[0]
# batch_size = 20
n_classes = 7
F_1 = 8
K = 8

X_in = Input(shape=(X.shape[-1],))
A_in = Input(shape=(batch_size,))

layer1 = GAT(F_1=F_1, K=K, last=False, input_shape=(X.shape[-1],))([X_in, A_in])
layer2 = GAT(F_1=n_classes, K=1, last=True)([layer1, A_in])

model = Model(inputs=[X_in, A_in], outputs=layer2)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              weighted_metrics=['accuracy'])
# print(model.summary())

validation_data = ([X, A], Y, val_idx)
model.fit([X, A], Y, validation_data=validation_data, sample_weight=train_idx, epochs=epochs, batch_size=batch_size, shuffle=False)

Train on 2708 samples, validate on 2708 samples
Epoch 1/300
2708/2708 [==============================] - 1s 547us/sample - loss: 0.1006 - accuracy: 0.1500 - val_loss: 0.3593 - val_accuracy: 0.1420
Epoch 2/300
2708/2708 [==============================] - 0s 109us/sample - loss: 0.1006 - accuracy: 0.2000 - val_loss: 0.3593 - val_accuracy: 0.1500
Epoch 3/300
2708/2708 [==============================] - 0s 102us/sample - loss: 0.1006 - accuracy: 0.2571 - val_loss: 0.3593 - val_accuracy: 0.1200
Epoch 4/300
2708/2708 [==============================] - 0s 96us/sample - loss: 0.1006 - accuracy: 0.3500 - val_loss: 0.3593 - val_accuracy: 0.1440
Epoch 5/300
2708/2708 [==============================] - 0s 93us/sample - loss: 0.1006 - accuracy: 0.4286 - val_loss: 0.3593 - val_accuracy: 0.1220
Epoch 6/300
2708/2708 [==============================] - 0s 95us/sample - loss: 0.1006 - accuracy: 0.5143 - val_loss: 0.3593 - val_accuracy: 0.1460
Epoch 7/300
2708/2708 [==============================] - 0s 9